Since the user experienced problems with R, further data cleaning shall be performed in Python for efficiency.

First, import relevant packages:

In [35]:
import pandas as pd
import numpy as np

Next, load the CSV file into a dataframe:

In [2]:
# CB_Data = pd.read_csv('CitiBike_data/202105-202204-citibike-trip-data.csv', low_memory=False)

Now, eliminate all instances of "joy rides", in which the rider docked the bike at the same station, either due to literally riding for fun or having a problem with the bike.

In [42]:
# joy_rides = CB_Data.loc[CB_Data['start_station_name'] == CB_Data['end_station_name']]

Int64Index([     258,      259,      260,      261,      262,      263,
                 264,      265,      266,      267,
            ...
            28814993, 28814994, 28814995, 28814996, 28814997, 28814999,
            28815001, 28815002, 28815003, 28815005],
           dtype='int64', length=1542162)

In [43]:
# CB_Data = CB_Data.drop(joy_rides.index)

Rule out the bikes that seem to come out of nowhere, if applicable

In [74]:
# ghost_bikes = CB_Data.loc[CB_Data['start_station_name'] == 'Unknown']

In [76]:
# CB_Data = CB_Data.drop(ghost_bikes.index)

Now, rule out the missing bikes.

In [77]:
# missing_bikes = CB_Data.loc[CB_Data['end_station_name'] == 'Unknown']

In [79]:
# CB_Data = CB_Data.drop(missing_bikes.index)

Finally, rule out the rides that were never completed, most likely to a rider changing their decision.

In [53]:
# no_rides = CB_Data.loc[CB_Data['duration'] == 0]

In [55]:
# CB_Data = CB_Data.drop(no_rides.index)

The issue in the R data analysis was the inability to calculate speed, due to null values in the duration. Therefore, as a final check, such a calculation shall be performed in order to assess the cleanliness of this dataframe so far:

In [57]:
# CB_Data['speed'] = CB_Data['distance'] / (0.0166667 * CB_Data['duration']) # in mph, convert min to hr at denom

So far, so good. Now, the station names need cleanup through an iterative process

In [87]:
nulls = CB_Data.loc[pd.isnull(CB_Data['CB_start_hood']) | pd.isnull(CB_Data['CB_end_hood'])]

In [88]:
nulls['start_station_name'].value_counts()

2 Ave & E 29 St                63472
8 Ave & W 16 St                63362
W 44 St & 11 Ave               56702
Forsyth St\t& Grand St         53086
6 Ave & W 45 St                53012
                               ...  
Lab - NYC                          7
E 6 St 2 Ave                       6
Avenue D & E 8 St                  3
Grand Concourse & E 161  St        2
Yankee Ferry Terminal              1
Name: start_station_name, Length: 1592, dtype: int64

In [89]:
nulls['end_station_name'].value_counts()

8 Ave & W 16 St           63465
2 Ave & E 29 St           62081
Forsyth St\t& Grand St    53520
6 Ave & W 45 St           53259
W 44 St & 11 Ave          52587
                          ...  
Brunswick St                  1
Hudson St & 4 St              1
StuyTown Depot                1
Jackson Square                1
Adams St & 2 St               1
Name: end_station_name, Length: 1672, dtype: int64

In [86]:
ques = ['Broadway & E 21 St','E 13 St & 2 Ave','Broadway & W 58 St','E 20 St & 2 Ave','5 Ave & E 72 St']
hood = ['Midtown East','East Village','Midtown West','Midtown East','Upper East Side']
boro = ['Manhattan','Manhattan','Manhattan','Manhattan','Manhattan']
zipped = zip(ques, hood, boro)

for q, h, b in zipped:
    CB_Data.CB_start_hood.loc[CB_Data.start_station_name == q] = h
    CB_Data.CB_start_boro.loc[CB_Data.start_station_name == q] = b
    CB_Data.CB_end_hood.loc[CB_Data.end_station_name == q] = h
    CB_Data.CB_end_boro.loc[CB_Data.end_station_name == q] = b

/Users/jchatterjee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [92]:
CB_Data.to_csv('CitiBike_data/202105-202204-citibike-trip-data.csv')